In [2]:
#! pip install jupysql pandas matplotlib duckdb-engine

In [3]:
import duckdb
import pandas as pd

%load_ext sql
    
conn = duckdb.connect("../data/antismash_db.duckdb")
%sql conn --alias duckdb

In [23]:
# How many genomes
genome = conn.sql("""
SELECT d.accession, g.*, t.* 
FROM antismash.genomes g 
JOIN antismash.taxa t ON g.tax_id = t.tax_id
JOIN antismash.dna_sequences d ON d.genome_id = g.genome_id;
""").df()

genome

,accession,genome_id,tax_id,bio_project,bio_sample,isolate_id,assembly_id,tax_id_1,ncbi_taxid,superkingdom,kingdom,phylum,CLASS,taxonomic_order,family,genus,species,strain,name
0,CP108339,1,37945,None,None,NaN,NBC_00023,37945,235987,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,jietaisiensis,NBC_00023,Streptomyces jietaisiensis NBC_00023
1,CP108825,2,37946,None,None,NaN,NBC_00860,37946,2975980,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Micromonosporales,Micromonosporaceae,Micromonospora,Unclassified,NBC_00860,Micromonospora sp. NBC 00860 NBC_00860
2,CP109017,3,37947,None,None,NaN,NBC_00670,37947,2975804,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,Unclassified,NBC_00670,Streptomyces sp. NBC 00670 NBC_00670
3,NBC_00640_1,4,37948,None,None,NaN,NBC_00640,37948,1961,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,virginiae,NBC_00640,Streptomyces virginiae NBC_00640
4,NBC_00640_2,4,37948,None,None,NaN,NBC_00640,37948,1961,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,virginiae,NBC_00640,Streptomyces virginiae NBC_00640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785,NBC_01669_4,1027,38971,None,None,NaN,NBC_01669,38971,2975909,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,Unclassified,NBC_01669,Streptomyces sp. NBC_01669 NBC_01669
1786,NBC_01669_5,1027,38971,None,None,NaN,NBC_01669,38971,2975909,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,Unclassified,NBC_01669,Streptomyces sp. NBC_01669 NBC_01669
1787,NBC_01669_8,1027,38971,None,None,NaN,NBC_01669,38971,2975909,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,Unclassified,NBC_01669,Streptomyces sp. NBC_01669 NBC_01669
1788,CP130727,1028,38972,None,None,NaN,NBC_00828,38972,2903678,Bacteria,Unclassified,Actinomycetota,Actinomycetes,Kitasatosporales,Streptomycetaceae,Streptomyces,Unclassified,NBC_00828,Streptomyces sp. NBC_00828 NBC_00828


In [84]:
table_schema = conn.sql("""
SELECT * 
FROM information_schema.columns 
WHERE table_schema = 'antismash';
"""
).df()
print("table", len(table_schema.table_name.unique()))
print("column", len(table_schema.column_name))

result = []
for t in table_schema.table_name.unique():
    query = f"SELECT COUNT(*) FROM antismash.{t};"
    query = conn.sql(query).fetchone()
    if query[0] > 500000:
        print(t, query)
    result.append(query[0])
print("rows", sum(result))

table 65
column 293
cdss (925127,)
cluster_compare_hits (3321736,)
genes (928430,)
pfam_domains (1230793,)
pfam_go_entries (878238,)
rows 9379701


In [81]:
max(result)

3321736

In [27]:
df_bgcflow = pd.read_csv("../data/samples_genbank_with_metadata_edited.csv")
df_bgcflow[~df_bgcflow.genome_id.isin(genome.assembly_id)].genome_id

17     NBC_01565
242    NBC_00443
673    NBC_01754
Name: genome_id, dtype: object

In [10]:
genome

┌───────────┬────────┬─────────────┬────────────┬───┬─────────────────┬───────────┬──────────────────────┐
│ genome_id │ tax_id │ bio_project │ bio_sample │ … │     species     │  strain   │         name         │
│   int32   │ int32  │   varchar   │  varchar   │   │     varchar     │  varchar  │       varchar        │
├───────────┼────────┼─────────────┼────────────┼───┼─────────────────┼───────────┼──────────────────────┤
│         1 │  37945 │ NULL        │ NULL       │ … │ jietaisiensis   │ NBC_00023 │ Streptomyces jieta…  │
│         2 │  37946 │ NULL        │ NULL       │ … │ Unclassified    │ NBC_00860 │ Micromonospora sp.…  │
│         3 │  37947 │ NULL        │ NULL       │ … │ Unclassified    │ NBC_00670 │ Streptomyces sp. N…  │
│         4 │  37948 │ NULL        │ NULL       │ … │ virginiae       │ NBC_00640 │ Streptomyces virgi…  │
│         5 │  37949 │ NULL        │ NULL       │ … │ jietaisiensis   │ NBC_00805 │ Streptomyces jieta…  │
│         6 │  37950 │ NULL        │ 

In [39]:
regions = conn.sql("""
SELECT * 
FROM antismash.regions r
JOIN antismash.rel_regions_types rrt ON r.region_id = rrt.region_id
JOIN antismash.bgc_types bt ON rrt.bgc_type_id = bt.bgc_type_id;
""").df()
regions.category.value_counts()

category
nrps          10003
other          9595
pks            9303
ripp           8266
terpene        5832
saccharide      164
Name: count, dtype: int64

In [31]:
regions.contig_edge.value_counts(normalize=True) * 100

contig_edge
False    98.984822
True      1.015178
Name: proportion, dtype: float64

In [40]:
regions

,region_id,accession,region_number,LOCATION,start_pos,end_pos,contig_edge,best_mibig_hit_acc,best_mibig_hit_description,best_mibig_hit_similarity,region_id_1,bgc_type_id,bgc_type_id_1,term,description,category
0,1,CP108339,1,[71679:125274],71679,125274,False,BGC0001841,detoxin P1/detoxin P2/detoxin P3,75.0,1,13,13,nrps,non-ribosomal peptide synthase,nrps
1,2,CP108339,2,[507164:528274],507164,528274,False,BGC0000187,asukamycin,20.0,2,60,60,furan,Furan,other
2,3,CP108339,3,[562605:572821],562605,572821,False,BGC0000518,informatipeptin,42.0,3,82,82,ripp-like,Fallback rule containing known RiPP-related pr...,ripp
3,4,CP108339,4,[935655:992641],935655,992641,False,BGC0000379,glycinocin A,6.0,4,13,13,nrps,non-ribosomal peptide synthase,nrps
4,5,CP108339,5,[1026002:1052817],1026002,1052817,False,BGC0000663,hopene,92.0,5,19,19,terpene,Terpene,terpene
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43158,30435,CP109333,15,[8037656:8064293],8037656,8064293,False,BGC0000663,hopene,84.0,30435,19,19,terpene,Terpene,terpene
43159,30436,CP109333,16,[8154217:8181766],8154217,8181766,False,None,None,NaN,30436,26,26,thiopeptide,Thio-linked RiPPs,ripp
43160,30436,CP109333,16,[8154217:8181766],8154217,8181766,False,None,None,NaN,30436,31,31,lap,Linear azol ( in ) e-containing peptides,ripp
43161,30437,CP109333,17,[8291627:8302425],8291627,8302425,False,BGC0000291,A54145,3.0,30437,82,82,ripp-like,Fallback rule containing known RiPP-related pr...,ripp


In [51]:
mibig_hits = regions[regions.best_mibig_hit_similarity > 80]
print(len(mibig_hits), len(mibig_hits) / len(regions)*100)
mibig_hits.best_mibig_hit_description.value_counts()[:10]

11661 27.016194425781343


best_mibig_hit_description
spore pigment                         1016
ectoine                                879
coelichelin                            588
hopene                                 550
desferrioxamin B/desferrioxamine E     545
alkylresorcinol                        529
ε-Poly-L-lysine                        517
geosmin                                497
albaflavenone                          307
isorenieratene                         296
Name: count, dtype: int64

In [55]:
mibig_hits = regions[regions.best_mibig_hit_similarity.isnull()]
print(len(mibig_hits), len(mibig_hits) / len(regions)*100)
mibig_hits.best_mibig_hit_description.value_counts()[:10]

5156 11.945416212960176


Series([], Name: count, dtype: int64)